# MiniZinc Interactive Examples

3 This notebook demonstrates constraint programming with MiniZinc using the `minizinc` Python package. 

**Running this notebook:**
- **Google Colab**: Click the "Open in Colab" button above. **Important**: You need to install MiniZinc first. Run this in a separate cell before running the setup:
  ```python
  !apt-get update && apt-get install -y minizinc
  ```
  Then restart the runtime (Runtime > Restart runtime) and continue.
- **Local Jupyter**: Install MiniZinc first (`brew install minizinc` on macOS, or download from [minizinc.org](https://www.minizinc.org/)), then `pip install minizinc`
- **Binder**: Alternative cloud environment (may require manual MiniZinc installation)

**Note**: This notebook uses the `minizinc` Python API directly. The `iminizinc` extension has compatibility issues with newer IPython versions.

## Setup

The next cell will check if MiniZinc is installed and import the necessary packages.


## Installation (Google Colab only)

**If you're running this on Google Colab**, run the cell below to install MiniZinc. Then restart the runtime (Runtime > Restart runtime) before continuing.

**If you're running locally**, skip this cell and make sure MiniZinc is installed on your system first.

In [7]:
# Install MiniZinc on Google Colab
# Run this cell, then restart the runtime (Runtime > Restart runtime)
!apt-get update && apt-get install -y minizinc

zsh:1: command not found: apt-get


In [8]:
# Install minizinc Python package
%pip install -q minizinc

import os
import subprocess
import shutil

# Check if MiniZinc binary is available
minizinc_path = shutil.which('minizinc')

if not minizinc_path:
    print("⚠ MiniZinc binary not found.")
    print("\n📋 To install on Google Colab, run this in a separate cell:")
    print("   !apt-get update && apt-get install -y minizinc")
    print("\n   Then restart the runtime (Runtime > Restart runtime) and run this cell again.")
    print("\n💡 For local Jupyter: Install MiniZinc first (e.g., 'brew install minizinc' on macOS)")
else:
    print(f"✓ MiniZinc found at: {minizinc_path}")
    try:
        result = subprocess.run([minizinc_path, '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"✓ {result.stdout.strip().split(chr(10))[0]}")
    except:
        pass

import minizinc
import asyncio

# Helper function to find an available solver
def get_solver(preferred=None):
    """Get an available solver, trying preferred first, then falling back to others."""
    # Check if driver is available
    if minizinc.default_driver is None:
        raise RuntimeError(
            "MiniZinc driver not found. Please ensure MiniZinc binary is installed.\n"
            "On Colab, this should be installed automatically. If not, try restarting the runtime."
        )
    
    driver = minizinc.default_driver
    tag_map = driver.available_solvers()
    
    # Try preferred solver first
    if preferred and preferred in tag_map and len(tag_map[preferred]) > 0:
        return minizinc.Solver.lookup(preferred)
    
    # Try common solvers in order of preference
    for solver_name in ["cbc", "coin-bc", "highs", "api"]:
        if solver_name in tag_map and len(tag_map[solver_name]) > 0:
            return minizinc.Solver.lookup(solver_name)
    
    # Fall back to first available
    if tag_map:
        first_tag = sorted(tag_map.keys())[0]
        if len(tag_map[first_tag]) > 0:
            return minizinc.Solver.lookup(first_tag)
    
    raise RuntimeError("No MiniZinc solvers available. Please install MiniZinc with solvers.")

# Note: Jupyter notebooks support top-level await, so we'll use solve_async() directly


Note: you may need to restart the kernel to use updated packages.
✓ MiniZinc found at: /opt/homebrew/bin/minizinc
✓ MiniZinc to FlatZinc converter, version 2.9.3


## Example 1: N-Queens Problem

Place N queens on an N×N chessboard so that no two queens attack each other.


In [9]:
n = 8

# MiniZinc model as a string
model_str = """
include "globals.mzn";
int: n;
set of int: Q = 1..n;

% q[i] is the column (1..n) of the queen in row i
array[Q] of var Q: q;

% No two queens in the same column
constraint all_different(q);

% No two queens on the same diagonal
constraint forall(i, j in Q where i < j) (
  abs(q[i] - q[j]) != j - i
);

solve satisfy;

output [
  if j = 1 then "\\n" else " " endif ++
  if fix(q[i]) = j then "Q" else "." endif
  | i in Q, j in Q
] ++ ["\\n", "Positions: ", show(q), "\\n"];
"""

# Create model and solve
model = minizinc.Model()
model.add_string(model_str)
model["n"] = n

# Get an available solver
solver = get_solver()
print(f"Using solver: {solver.name}")
instance = minizinc.Instance(solver, model)

# Solve and display result (using async for Jupyter compatibility)
result = await instance.solve_async()
print(result)


Using solver: COIN-BC

. . . . . . Q .
. . Q . . . . .
Q . . . . . . .
. . . . . Q . .
. . . . . . . Q
. . . . Q . . .
. Q . . . . . .
. . . Q . . . .
Positions: [7, 3, 1, 6, 8, 5, 2, 4]



## Example 2: Employee Shift Scheduling

Assign employees to shifts with fairness constraints.


In [3]:
E = 5  # number of employees
D = 7  # number of days
S = 3  # shifts per day
max_shifts = 5  # maximum shifts per employee per week

# MiniZinc model as a string
model_str = """
include "globals.mzn";

int: E;
int: D;
int: S;
int: max_shifts_per_employee;

set of int: Employees = 1..E;
set of int: Days = 1..D;
set of int: Shifts = 1..S;

% Decision variables: x[e,d,s] = 1 if employee e works shift s on day d
array[Employees, Days, Shifts] of var 0..1: x;

% Each shift needs exactly 1 employee
constraint forall(d in Days, s in Shifts) (
  sum(e in Employees)(x[e,d,s]) = 1
);

% Each employee works at most one shift per day
constraint forall(e in Employees, d in Days) (
  sum(s in Shifts)(x[e,d,s]) <= 1
);

% Weekly cap for fairness
constraint forall(e in Employees) (
  sum(d in Days, s in Shifts)(x[e,d,s]) <= max_shifts_per_employee
);

solve satisfy;

output [
  "Employee " ++ show(e) ++ " works: " ++
  concat([ if fix(x[e,d,s]) = 1 then "Day " ++ show(d) ++ " Shift " ++ show(s) ++ ", " else "" endif
         | d in Days, s in Shifts ]) ++ "\\n"
  | e in Employees
];
"""

# Create model and solve
model = minizinc.Model()
model.add_string(model_str)
model["E"] = E
model["D"] = D
model["S"] = S
model["max_shifts_per_employee"] = max_shifts

# Get an available solver
solver = get_solver()
print(f"Using solver: {solver.name}")
instance = minizinc.Instance(solver, model)

# Solve and display result (using async for Jupyter compatibility)
result = await instance.solve_async()
print(result)


Using solver: COIN-BC
Employee 1 works: Day 3 Shift 1, Day 6 Shift 2, 
Employee 2 works: Day 3 Shift 3, Day 4 Shift 1, Day 5 Shift 2, Day 7 Shift 1, 
Employee 3 works: Day 1 Shift 3, Day 2 Shift 2, Day 4 Shift 3, Day 5 Shift 1, Day 7 Shift 3, 
Employee 4 works: Day 1 Shift 1, Day 2 Shift 1, Day 4 Shift 2, Day 6 Shift 3, Day 7 Shift 2, 
Employee 5 works: Day 1 Shift 2, Day 2 Shift 3, Day 3 Shift 2, Day 5 Shift 3, Day 6 Shift 1, 



## Example 3: Finding All Solutions

Some solvers support finding all solutions using the `-a` flag. Note that not all solvers support this feature.


In [4]:
n = 4

# MiniZinc model as a string
model_str = """
include "globals.mzn";
int: n;
set of int: Q = 1..n;
array[Q] of var Q: q;
constraint all_different(q);
constraint forall(i, j in Q where i < j) (
  abs(q[i] - q[j]) != j - i
);
solve satisfy;
output [show(q), "\\n"];
"""

# Create model
model = minizinc.Model()
model.add_string(model_str)
model["n"] = n

# Get an available solver
solver = get_solver()
print(f"Using solver: {solver.name}")
instance = minizinc.Instance(solver, model)

# Try to find all solutions (using async for Jupyter compatibility)
print("All solutions for N-Queens with n =", n, ":")
try:
    solutions = await instance.solve_async(all_solutions=True)
    # Note: all_solutions returns a list of solutions
    if isinstance(solutions, list):
        for i, solution in enumerate(solutions, 1):
            print(f"Solution {i}: {solution['q']}")
    else:
        # If it's a single solution or different format
        print(f"Solution: {solutions['q']}")
except NotImplementedError:
    print(f"Note: The solver '{solver.name}' does not support finding all solutions.")
    print("Finding a single solution instead:")
    result = await instance.solve_async()
    print(f"Solution: {result['q']}")
    print("\nTo find all solutions, you may need to use a different solver that supports the -a flag.")
except Exception as e:
    print(f"Error finding all solutions: {e}")
    print("Finding a single solution instead:")
    result = await instance.solve_async()
    print(f"Solution: {result['q']}")


Using solver: COIN-BC
All solutions for N-Queens with n = 4 :
Note: The solver 'COIN-BC' does not support finding all solutions.
Finding a single solution instead:
Solution: [3, 1, 4, 2]

To find all solutions, you may need to use a different solver that supports the -a flag.
